In [3]:
//#r "nuget: xunit, 2.8.1"
using System.Threading;
using Xunit;
using System.Collections.Concurrent;
using System.Threading.Tasks;

class Message
{
    static int messageHandled = 0;
    int id;
    public Message()
    {
        id = Thread.CurrentThread.ManagedThreadId;
    }
    public void Handle()
    {
        Interlocked.Increment(ref messageHandled);
        Assert.NotEqual(id, Thread.CurrentThread.ManagedThreadId);
    }
    public static void Check()
    {
        Assert.Equal(200, messageHandled);
    }
}
var q1 = new BlockingCollection<Message>(2); //очередь первого потока
var q2 = new BlockingCollection<Message>(2); //очередь второго потока

bool f1 = false;
bool f2 = false;
Thread t1 = new Thread(() => {
    for (int i = 0; i < 100; i++)
    {
        Message m = q1.Take();
        m.Handle();
        q2.Add(m);
        if (i == 99) f1 = true;
    }
});
Thread t2 = new Thread(() => {
    for (int i = 0; i < 100; i++)
    {
        Message m = q2.Take();
        m.Handle();
        if (i < 99) q1.Add(m);
        if (i == 99) f2 = true;
    }
});
Message nm = new Message();
q1.Add(nm);
t1.Start();
t2.Start();
Task checkAll  = new Task(() => {
    Assert.Equal(0, q1.Count);
    Assert.Equal(0, q2.Count);
    Message.Check();
    Console.WriteLine("Checked");
});
while (!f2 & !f1) checkAll.Wait(10); 
checkAll.Start();

Checked
